In [ ]:
import cmipld
import importlib
from collections import OrderedDict


repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

whoami = cmipld.reverse_mapping()[io_url]
print('-'*50)
print(f'Parsing repo: {whoami}')
print(f'Location: {repo_url}')
print(f'Github IO link: {io_url}')
print('-'*50)

'''
To keep this set of functions generic, we create separate files containing all relevant functions for a repository to be processed. 
When running this script we determine the parent repository, and consequently its prefix. This is then used to grab all relevant functions for processing. 

Example usage: getattr(this,'activity')({'activity':[{"label":1,"id":'one'}]})

'''
# Dynamically import the target module and assign it to a variable
module_path = f"cmipld.cvs.{whoami}" # go to CMIPLD/CVs/<value of whoami>.py for functions
this = importlib.import_module(module_path)



In [ ]:
# Set up offline override for the current repository

from cmipld.utils.offline import LD_server

ldpath = cmipld.utils.git.ldpath()


# create a temporary directory containing (a copy of) the current repository
# localserver = LD_server(copy = [[ldpath,'current_repo']])
repos= { 
        'https://wcrp-cmip.github.io/WCRP-universe/': 'universal',
        'https://wcrp-cmip.github.io/MIP-variables/': 'variables',
        'https://wcrp-cmip.github.io/CMIP6Plus_CVs/': 'cmip6plus'
        }

localserver = LD_server(repos = repos.items(),copy = [[ldpath,whoami]],override='y')

# localserver = LD_server(copy = [[ldpath,'cmip6plus']])


localhost = localserver.start_server(8089)


In [ ]:
cmipld.processor.replace_loader(localhost,[list(i) for i in repos.items()]) 

In [4]:

CV = OrderedDict()
cmipld.processor.clear_cache()


es_data = {}

In [ ]:


es_data['project'] = project = cmipld.processor.EmbeddedFrame(f'{io_url}project/graph.jsonld') 

for item in [
    'repo',
    'mip-era',
    'license',
    'activity',
    'drs',
    'product',
    'required-global-attributes',
    'tables',
    ]:
    
    print(item)
    # CV[item] = getattr(locals(),item)
    # ld  = cmipld.processor.expand_document(f'{io_url}project/{item}.json', depth=3)[0]
    # functions may be autogenrated in (reponame).py under the CVs dir. 
    
    result = project.frame({'id':f'{io_url}project/{item}-list'})
    from pprint import pprint 
    # pprint(result)
    
    print(item,getattr(this,item))
    CV[item] = getattr(this,item)(result[0])
    
CV

In [ ]:
# project.corpus['@graph'][4]
result

In [ ]:
localserver.copy_existing_repos([[ldpath,whoami]],override=True)


cmipld.processor.clear_cache()

item = 'mip-era'
print(f'{io_url}project/{item}.json')

url = f'{io_url}project/{item}-list'

result = project.frame({'@id':url})
# ,'@context':url})
# result = project.frame({'@type':f'wcrp:{item}'})

item,result

In [ ]:
from pprint import pprint
# pprint(cmipld.jsonld.frame(project.corpus["@graph"][2],{"id":url,"@context":project.context}))

pprint(cmipld.jsonld.compact(project.corpus['@graph'][2],url))



In [ ]:
project.context

In [ ]:
project.corpus["@graph"][2]


In [ ]:
# adding the index properties
print('index')
CV.update(cmipld.processor.get(f'{io_url}/project/index.json')[0]['index'])

In [ ]:
print('make a file for relevant realms and relevant realms')

for universe_item in ['realm','frequency']:
    # 'grid-label''nominal'
    print(universe_item)
    path = f'{cmipld.mapping["universal"]}{universe_item}/graph.jsonld'
    ld  = cmipld.jsonld.compact(path,path)['@graph']
    CV[universe_item] = getattr(this,universe_item)(ld)

In [ ]:
# path = f'{cmipld.mapping["universal"]}resolution/graph.jsonld'
# data  = cmipld.jsonld.compact(path,path)['@graph']

# CV['nominal_resolution'] = [f"{r['value']} {r['unit']}" for r in sorted(data, key=lambda x: float(x['value']) if x['value'].replace('.', '', 1).isdigit() else float('inf'))]

print('make a file for relevant nominal resolution')

In [ ]:
# url = cmipld.processor.resolve_prefix(file)

# loads the experiments graph and relevant dependancies. 
# cmipld.processor.clear_cache()
# localserver.copy_existing_repos([[ldpath,whoami]],override=True)
print(ldpath,whoami)

es_data = {}

for value in ['experiment', 'source']:
    print(value)
    es_data[value] = cmipld.processor.EmbeddedFrame(f'{io_url}{value}/graph.jsonld') 
    result = es_data[value].frame({'@type':[f'wcrp:{value}']})
    
    # CV[value+'_id'] = getattr(this,value)(result)
# 

value, es_data[value].dependencies

In [ ]:
# CV[value+'_id']
result[0]

In [ ]:
CV

In [ ]:
ls 
